In [4]:
import numpy as np
import matplotlib.pyplot as plt
import gym
import pickle #allows to serialize and deserialize Python objects.
from mpl_toolkits.mplot3d import Axes3D
from tqdm import trange,tqdm #allows to create progress bars
import os

#change the paths to the correct paths on your computer
directory_path_params = r"C:\Users\Callens.R\Documents\Kms\1MA sem 2\2. DS425 Intelligent decision support methods\Project weemaes_callens\DS425-Blackjack\data\params"

In this code the winrate is calculated for a range of reward functions. 

In [5]:
env = gym.make('Blackjack-v1')

In [6]:
# Define the state space
player_sum_space = range(4, 22)  # possible player hand values (4, 5, ..., 20, 21)
dealer_card_space = range(1, 11)  # possible dealer up card values (2, 3, ..., 10, 11)
usable_ace_space = [False, True]  # whether the player has a usable ace

state_space = []
for player_sum in player_sum_space:
    for dealer_card in dealer_card_space:
        for usable_ace in usable_ace_space:
            state_space.append((player_sum, dealer_card, usable_ace))

# This creates an array of length 306, which is the number of possible states in the game of blackjack. 
# Each state is a tuple of the player’s sum, the dealer’s card, and whether the player has a usable ace.

# Define the action space
action_space = [0, 1]  # hit or stick
# Get the size of the state and action spaces
num_states = len(state_space)
num_actions = len(action_space)

Change the reward function and chek the influence on the winrate. The value of the number of episodes of the q table, games and hyperparameters stay cosntant

In [10]:
# Hyperparameters
alpha = 0.05  # Learning rate
gamma = 0.99  # Discount factor
epsilon = 0.1  # Epsilon-greedy parameter
num_games = 100000
num_draws = 0

rewarddraw_range = np.arange(-1,1,0.1)
rewardwin_range = np.arange(-4,20,1)
rewardloss_range = np.arange(-20,4,1)

# Initialize arrays to store win rate for each combination of hyperparameters
win_rates = np.zeros((rewarddraw_range.size, rewardwin_range.size, rewardloss_range.size))

# Vary num_episodes and compute winrate
num_episodes = 1000000
winrates = []
for i, rewarddraw in enumerate(rewarddraw_range):
    for j, rewardwin in enumerate(rewardwin_range):
        for k, rewardloss in enumerate(rewardloss_range):

            # Check if a file already exists for this Q-table
            filename = f"{directory_path_params}/episodes{num_episodes}_q_table_alpha{alpha}_gamma{gamma}_epsilon{epsilon}_RD{rewarddraw}_RW{rewardwin}_RL{rewardloss}.pkl"
            if os.path.exists(filename):
                #print(f"{filename} already exists, loading...")
                with open(filename, 'rb') as f:
                    Q_table = pickle.load(f)
            else:
                Q_table = np.zeros((len(state_space), len(action_space)))
                for episode in range(num_episodes):
                    state = env.reset()[0]
                    done = False
                    while not done:
                        # Epsilon-greedy action selection
                        if np.random.random() < epsilon:
                            action = np.random.choice(action_space)
                        else:
                            action = np.argmax(Q_table[state_space.index(state)])

                        # Take the action
                        next_state, reward, done, terminal, dic = env.step(action)
                        
                        if reward == 0:
                            rewardcalc = rewarddraw
                        elif reward == 1:
                            rewardcalc = rewardwin
                        elif reward == -1:
                            rewardcalc = rewardloss
                        # Update Q-value if you busted, necessary because Q_table(next_state) does not exist if you bust
                        if next_state not in state_space:
                            Q_table[state_space.index(state)][action] += alpha * (rewardcalc - Q_table[state_space.index(state)][action])
                            break

                        # Update Q-value for current state-action pair
                        Q_table[state_space.index(state)][action] += alpha * (rewardcalc + gamma * np.max(Q_table[state_space.index(next_state)]) - Q_table[state_space.index(state)][action])

                        state = next_state

                    # Save the Q-table to a file using pickle
                    with open(filename, 'wb') as f:
                        pickle.dump(Q_table, f)

            # Evaluate the performance of the Q-table by playing num_games games
            num_wins = 0
            num_losses = 0
            num_draws = 0
            for i in range(num_games):
                state = env.reset()[0]
                done = False
                while not done:
                    action = np.argmax(Q_table[state_space.index(state)])
                    next_state, reward, done, terminal, dic = env.step(action)
                    state = next_state
                    if done and reward == 1:
                        num_wins += 1
                    elif done and reward == -1:
                        num_losses += 1
                    elif done and reward == 0:
                        num_draws += 1

            win_rate = num_wins / (num_games - num_draws)
            win_rates[i, j, k] = win_rate

# Visualize results
draw_grid, win_grid, loss_grid = np.meshgrid(rewarddraw_range, rewardwin_range, rewardloss_range, indexing='ij')
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
sc = ax.scatter(draw_grid, win_grid, loss_grid, c=win_rates.flatten(), cmap='coolwarm')
ax.set_xlabel('Reward Draw')
ax.set_ylabel('Reward Win')
ax.set_zlabel('Reward Loss')

# Add colorbar
cbar = fig.colorbar(sc)
cbar.ax.set_ylabel('Win Rate', rotation=270, labelpad=10)

plt.show()

c:\Users\Callens.R\AppData\Local\Programs\Python\Python311\Lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


KeyboardInterrupt: 